In [ ]:
import os
os.chdir("/home/jakob/doktor/projects/EnsembleUncertainty/code")
"""Learing "logit" distribution in regression example"""
import logging
import zipfile
import urllib.request
from pathlib import Path
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.autograd import Variable
from torch.optim import Optimizer
from torch.optim.sgd import SGD
from torchvision import datasets, transforms
from torchvision.utils import make_grid
from sklearn.model_selection import KFold
from src.dataloaders.uci import uci_base, wine, bost
from src import metrics
from src import utils
from src.ensemble import simple_regressor
from src import loss as custom_loss



LOGGER = logging.getLogger(__name__)
EXPERIMENT_NAME = "red_regression_logits"

# Settings
class Args():
    pass
args = Args()
args.seed = 1
args.gpu = False
args.log_dir = Path("./logs")
args.log_level = logging.INFO
args.retrain = True

args.num_ensemble_members=1
args.num_epochs=1
args.lr = 0.01

# General constructs
rmse = metrics.Metric(name="RMSE", function=metrics.root_mean_squared_error)
BATCH_SIZE = 32
torch.cuda.device(0)
torch.cuda.get_device_name(torch.cuda.current_device())
device = torch.device("cuda")
%load_ext autoreload
%autoreload 2
%config InlineBackend.figure_format = 'svg'

In [ ]:
def to_variable(var=(), cuda=True, volatile=False):
    out = []
    for v in var:
        
        if isinstance(v, np.ndarray):
            v = torch.from_numpy(v).type(torch.FloatTensor)

        if not v.is_cuda and cuda:
            v = v.cuda()

        if not isinstance(v, Variable):
            v = Variable(v, volatile=volatile)

        out.append(v)
    return out

In [ ]:
def get_loss_and_rmse(network, loss_function, x, y, num_samples, mean_shift=None, std_scale=None):
    x, y = to_variable(var=(x, y), cuda=True)

    means, stds = [], []
    for i in range(num_samples):
        logits = network.forward(x)
        output = network.transform_logits(logits)
        means.append(output[0])
        stds.append(output[1])
    means, stds = torch.cat(means, dim=1), torch.cat(stds, dim=1)
    mean = means.mean(dim=-1)[:, None]
    std = ((means.var(dim=-1) + stds.mean(dim=-1)**2)**0.5)[:, None]

    if mean_shift is not None and std_scale is not None:
        mean_shift = torch.tensor(mean_shift).float().cuda()
        std_scale = torch.tensor(std_scale).float().cuda()
        mean = mean * std_scale + mean_shift
        y = y * std_scale + mean_shift
        std *= std_scale

    loss = loss_function((mean, std), y)

    rmse = ((mean - y)**2).mean()**0.5

    return loss.detach().cpu(), rmse.detach().cpu()

# UCI datasets

In [ ]:
def train_mc_dropout(data,
                     drop_prob,
                     n_splits,
                     num_epochs,
                     num_units,
                     learn_rate,
                     weight_decay,
                     log_every,
                     num_samples,
                     batch_size):
    
    kf = KFold(n_splits=n_splits)
    in_dim = data.shape[1] - 1
    train_logliks, test_logliks = [], []
    train_rmses, test_rmses = [], []
    my_mse = metrics.Metric(name="MSE", function=metrics.mean_squared_error)
    
    for j, idx in enumerate(kf.split(data)):
        print('FOLD %d:' % j)
        train_index, test_index = idx

        x_train, y_train = data[train_index, :in_dim], data[train_index, in_dim:]
        x_test, y_test = data[test_index, :in_dim], data[test_index, in_dim:]

        x_means, x_stds = x_train.mean(axis = 0), x_train.var(axis = 0)**0.5
        y_means, y_stds = y_train.mean(axis = 0), y_train.var(axis = 0)**0.5

        x_train = (x_train - x_means) / x_stds
        y_train = (y_train - y_means) / y_stds

        x_test = (x_test - x_means) / x_stds
        y_test = (y_test - y_means) / y_stds
                
        dataloader = uci_base.uci_dataloader(x_train, y_train, batch_size)

        network = simple_regressor.Model(layer_sizes=[in_dim, num_units, 2],
                                         device=device,
                                         variance_transform=utils.variance_linear_asymptote(),
                                         loss_function=custom_loss.gaussian_neg_log_likelihood_1d)
                            
        network.optimizer = torch.optim.SGD(network.parameters(),
                                    lr=learn_rate,
                                    weight_decay=weight_decay)

        losses = []
        fit_loss_train = np.zeros(num_epochs)
        for i in range(num_epochs):
            if batch_size is not None:
                #loss = train_epoch(net, dataloader)
                loss = network._train_epoch(dataloader)
                
        mean_shift = None
        std_scale = None
        #mean_shift = y_means
        #std_scale = y_stds
        train_loss, train_rmse = get_loss_and_rmse(network,
                                                   network.loss,
                                                   x_train,
                                                   y_train,
                                                   num_samples=num_samples,
                                                   mean_shift=mean_shift,
                                                   std_scale=std_scale)
        
        test_loss, test_rmse = get_loss_and_rmse(network,
                                                 network.loss,
                                                 x_test,
                                                 y_test,
                                                 num_samples=num_samples,
                                                 mean_shift=mean_shift,
                                                 std_scale=std_scale)
        #my_mse.update(net.network(x_test), y_test)
        train_logliks.append((train_loss.cpu().data.numpy()/len(x_train) + np.log(y_stds)[0]))
        test_logliks.append((test_loss.cpu().data.numpy()/len(x_test) + np.log(y_stds)[0]))

        train_rmses.append(y_stds[0]*train_rmse.cpu().data.numpy())
        test_rmses.append(y_stds[0]*test_rmse.cpu().data.numpy())
        print('Train loss: %6.3f Test loss: %6.3f RMSE: %.3f' % 
              (loss/len(x_train), test_loss/len(x_test), test_rmse*y_stds[0]))


    print('Train log. lik. = %6.3f +/- %6.3f' % (-np.array(train_logliks).mean(),
                                                 np.array(train_logliks).var()**0.5))
    print('Test  log. lik. = %6.3f +/- %6.3f' % (-np.array(test_logliks).mean(),
                                                 np.array(test_logliks).var()**0.5))
    print('Train RMSE      = %6.3f +/- %6.3f' % (np.array(train_rmses).mean(),
                                                 np.array(train_rmses).var()**0.5))
    print('Test  RMSE      = %6.3f +/- %6.3f' % (np.array(test_rmses).mean(),
                                                 np.array(test_rmses).var()**0.5))
    #print('Test  RMSE      = %6.3f +/- %6.3f' % (np.array(my_rmses).mean(), np.array(my_rmses).var()**0.5))

# Red wine dataset

In [ ]:
wine_data = wine.WineData("data/uci/wine/winequality-red.csv").data


In [ ]:
train_mc_dropout(data=wine_data,
                       drop_prob=0.0,
                       num_epochs=40,
                       n_splits=10,
                       num_units=50,
                       learn_rate=1e-4,
                       weight_decay=0.0, #1e-1/len(data)**0.5,
                       num_samples=2,
                       log_every=50,
                       batch_size=1000)

# Housing dataset

In [ ]:
bost_data = bost.BostonData("data/uci/bost/housing.data").data

In [ ]:
net = train_mc_dropout(data=bost_data,
                       drop_prob=0.0,
                       num_epochs=40,
                       n_splits=10,
                       num_units=50,
                       learn_rate=1e-4,
                       weight_decay=0.0, #1e-1/len(data)**0.5,
                       num_samples=20,
                       log_every=50,
                       batch_size=1411)